In [1]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

/Users/daphnehansell/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")



In [3]:

situation_new_york = {
  "people": {
    "you": {
      "age": {
        "2026": 30
      }
    },
    "your partner": {
      "age": {
        "2026": 30
      }
    },
    "your first dependent": {
      "age": {
        "2026": 3
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent"
      ]
    }
  },
  "households": {
   "your household": {
     "members": [
       "you",
       "your partner", 
       "your first dependent"  # All live in the same household
     ],
     "state_name": {
       "2026": "NY"  # Located in New York state
     },
    "county_fips": {
      "2026": "36061"
    }
   }
 },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2026": 1
      }
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [4]:
situation_texas = {
  "people": {
    "you": {
      "age": {
        "2026": 25
      }
    },
    "your partner": {
      "age": {
        "2026": 28
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2026": "TX"
      },
      "county_fips": {
        "2026": "48015"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}



In [5]:
simulation_new_york = Simulation(
    situation=situation_new_york,
)
simulation_texas = Simulation(
    situation=situation_texas,
)
reformed_simulation_new_york = Simulation(
    situation=situation_new_york,
    reform=reform,
)
reformed_simulation_texas = Simulation(
    situation=situation_texas,
    reform=reform,
)

In [6]:
import copy
import pandas as pd
from policyengine_us import Simulation

# ------------------------------------------------------------------
# 1.  Convenience: build & run a one-income Texas couple simulation
# ------------------------------------------------------------------
def calc_ptc_for_income(base_situation: dict, income: float, *, use_reform=False):
    """
    Return ACA PTC (household-level) for the given income.

    Parameters
    ----------
    base_situation : dict
        The original `situation_texas` dict from the notebook.
    income : float
        Total household employment income to test (USD, annual).
    use_reform : bool, default False
        If True, applies the IRA-style `reform` object.

    Returns
    -------
    float
        PTC in dollars for period 2026.
    """
    # Deep-copy so we don’t mutate the original dict
    sit = copy.deepcopy(base_situation)

    # Remove axes so we run a single-point simulation
    sit.pop("axes", None)

    # Split income 50/50 between the two adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {"2026": income / 2}

    # Run the simulation (baseline or reform)
    sim = Simulation(situation=sit, reform=reform if use_reform else None)
    # Household-level PTC for 2026 comes back as a 1-element array
    return sim.calculate("aca_ptc", map_to="household", period=2026)[0]

# ------------------------------------------------------------------
# 2.  Pull the three requested incomes
# ------------------------------------------------------------------
targets = {
    "138 % FPL ($29,187)": 29_187,
    "300 % FPL ($63,450)": 63_450,
    "400 % FPL ($84,600)": 84_600,
}

rows = []
for label, inc in targets.items():
    rows.append(
        {
            "income_label": label,
            "income_usd": inc,
            "ptc_baseline": calc_ptc_for_income(situation_texas, inc, use_reform=False),
            "ptc_ira_reform": calc_ptc_for_income(situation_texas, inc, use_reform=True),
        }
    )

ptc_df = pd.DataFrame(rows)
ptc_df


,income_label,income_usd,ptc_baseline,ptc_ira_reform
0,"138 % FPL ($29,187)",29187,9128.747070,10090.201172
1,"300 % FPL ($63,450)",63450,4062.451172,6283.201172
2,"400 % FPL ($84,600)",84600,0.000000,2899.201172


In [18]:
import copy
import pandas as pd
from policyengine_us import Simulation

# ------------------------------------------------------------------
# 1.  Helper: run a one-point simulation and collect metrics
# ------------------------------------------------------------------
def get_metrics_for_income(base_situation: dict, income: float):
    """
    Returns baseline & reform PTC plus baseline Medicaid and CHIP metrics
    for a New York family of three at the specified annual income.

    Parameters
    ----------
    base_situation : dict
        Your `situation_ny` dictionary.
    income : float
        Total household employment income to test (USD, annual).

    Returns
    -------
    dict  with keys
        ptc_baseline       – ACA PTC in baseline
        ptc_ira_reform     – ACA PTC under IRA-style reform
        medicaid_cost      – household Medicaid benefit (baseline)
        per_capita_chip    – CHIP benefit ÷ household size (baseline)
    """
    # ---------------- Copy & inject the income --------------------
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)                # single-point sim only

    # Split income equally between both adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {"2026": income / 2}

    hh_size = len(sit["people"])

    # ---------------- Run simulations ----------------------------
    sim_base   = Simulation(situation=sit)
    sim_reform = Simulation(situation=sit, reform=reform)

    # ---------------- Pull variables -----------------------------
    # ACA PTC
    ptc_base   = sim_base.calculate("aca_ptc",   map_to="household", period=2026)[0]
    ptc_reform = sim_reform.calculate("aca_ptc", map_to="household", period=2026)[0]

    # Medicaid benefit (adult or child)
    medicaid_cost = sim_base.calculate("medicaid_per_capita_cost", map_to="household", period=2026)[0]

    # CHIP benefit – variable names differ by PE-US version:
    #   * If your build has `chip_cost`, use that.
    #   * Otherwise use `chip` (total CHIP dollars) or adjust as needed.
    chip_total = sim_base.calculate("per_capita_chip", map_to="household", period=2026)[0]
    per_capita_chip = chip_total / hh_size if hh_size else 0

    return dict(
        ptc_baseline     = ptc_base,
        ptc_ira_reform   = ptc_reform,
        medicaid_cost    = medicaid_cost,
        per_capita_chip  = per_capita_chip,
    )

# ------------------------------------------------------------------
# 2.  Income targets (family of 3, 2026 FPL thresholds you supplied)
# ------------------------------------------------------------------
targets_ny = {
    "154 % FPL ($41,041)" : 41_041,
    "200 % FPL ($53,300)" : 53_300,
    "300 % FPL ($79,950)": 79_950,
    "405 % FPL ($107,933)": 107_933,
}

rows = []
for label, inc in targets_ny.items():
    metrics = get_metrics_for_income(situation_new_york, inc)
    rows.append(
        dict(income_label = label, income_usd = inc, **metrics)
    )

ny_ptc_df = pd.DataFrame(rows)
ny_ptc_df


,income_label,income_usd,ptc_baseline,ptc_ira_reform,medicaid_cost,per_capita_chip
0,"154 % FPL ($41,041)",41041,0.0,0.000000,16480.696289,0.000000
1,"200 % FPL ($53,300)",53300,0.0,0.000000,12930.671875,829.929932
2,"405 % FPL ($107,933)",107933,0.0,12268.548828,0.000000,829.929932


['you', 'your partner', 'your first dependent']


In [7]:
household_income_new_york = simulation_new_york.calculate("employment_income", map_to="household", period=2026)
baseline_new_york_per_capita_chip = simulation_new_york.calculate("per_capita_chip", map_to="household", period=2026)
baseline_new_york_aca_ptc = simulation_new_york.calculate("aca_ptc", map_to="household", period=2026)
baseline_new_york_medicaid_cost = simulation_new_york.calculate("medicaid_per_capita_cost", map_to="household", period=2026)
baseline_new_york_net_income_including_health_benefits = simulation_new_york.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)


reform_new_york_per_capita_chip = reformed_simulation_new_york.calculate("per_capita_chip", map_to="household", period=2026)
reform_new_york_aca_ptc = reformed_simulation_new_york.calculate("aca_ptc", map_to="household", period=2026)
reform_new_york_medicaid_cost = reformed_simulation_new_york.calculate("medicaid_per_capita_cost", map_to="household", period=2026)
reform_new_york_net_income_including_health_benefits = reformed_simulation_new_york.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)





# Get household-level values for Texas
household_income_texas = simulation_texas.calculate("employment_income", map_to="household", period=2026)
baseline_texas_per_capita_chip = simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
baseline_texas_aca_ptc = simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
baseline_texas_medicaid_cost = simulation_texas.calculate("medicaid_per_capita_cost", map_to="household", period=2026)
baseline_texas_net_income_including_health_benefits = simulation_texas.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)


reform_texas_per_capita_chip = reformed_simulation_texas.calculate("per_capita_chip", map_to="household", period=2026)
reform_texas_aca_ptc = reformed_simulation_texas.calculate("aca_ptc", map_to="household", period=2026)
reform_texas_medicaid_cost = reformed_simulation_texas.calculate("medicaid_per_capita_cost", map_to="household", period=2026)
reform_texas_net_income_including_health_benefits = reformed_simulation_texas.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)


# Calculate total benefits for each scenario
baseline_new_york_total = [sum(x) for x in zip(baseline_new_york_per_capita_chip, baseline_new_york_aca_ptc, baseline_new_york_medicaid_cost)]
reform_new_york_total = [sum(x) for x in zip(reform_new_york_per_capita_chip, reform_new_york_aca_ptc, reform_new_york_medicaid_cost)]

baseline_texas_total = [sum(x) for x in zip(baseline_texas_per_capita_chip, baseline_texas_aca_ptc, baseline_texas_medicaid_cost)]
reform_texas_total = [sum(x) for x in zip(reform_texas_per_capita_chip, reform_texas_aca_ptc, reform_texas_medicaid_cost)]



In [8]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

In [9]:
# Create NY graph
fig_new_york = go.Figure()

# Add baseline traces (solid lines)
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=baseline_new_york_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_new_york.add_trace(go.Scatter(
    x=household_income_new_york, 
    y=reform_new_york_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_new_york.update_layout(
    title='New York Household (Family of 3) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Create Texas graph
fig_texas = go.Figure()

# Add baseline traces (solid lines)
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_per_capita_chip, 
    mode='lines', 
    name='CHIP (Baseline)', 
    line=dict(color=GRAY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Baseline)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

# Add reform traces (dotted lines)
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_per_capita_chip, 
    mode='lines', 
    name='CHIP (Reform)', 
    line=dict(color=GRAY, width=2, dash='dot')
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_medicaid_cost, 
    mode='lines', 
    name='Medicaid (Reform)', 
    line=dict(color=TEAL_ACCENT, width=2, dash='dot')
))

# Add total lines
fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=baseline_texas_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_texas.add_trace(go.Scatter(
    x=household_income_texas, 
    y=reform_texas_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_texas.update_layout(
    title='Texas Household (Couple) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Apply your format_fig function if it exists
# If you don't have this function defined, you can remove these lines
fig_new_york = format_fig(fig_new_york)
fig_texas = format_fig(fig_texas)

# Display the figures
fig_new_york.show()
fig_texas.show()

In [10]:
#House hold net income graphs
import plotly.graph_objects as go

# ---------- NY fam  ----------
fig_ny = go.Figure()

# Baseline (solid)
fig_ny.add_trace(go.Scatter(
    x=household_income_new_york,
    y=baseline_new_york_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_ny.add_trace(go.Scatter(
    x=household_income_new_york,
    y=reform_new_york_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_ny.update_layout(
    title='New York Household (Family of 3) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_ny = format_fig(fig_ny)

fig_ny.show()


In [11]:
# ---------- texas couple ----------
fig_tx = go.Figure()

# Baseline (solid)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=baseline_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)        # use your palette constant
))

# Reform (dotted)
fig_tx.add_trace(go.Scatter(
    x=household_income_texas,
    y=reform_texas_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_tx.update_layout(
    title='Texas Household (Married Couple) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_tx = format_fig(fig_tx)

fig_tx.show()


In [12]:
# ---------- Pull the inputs ----------
household_income_new_york = simulation_new_york.calculate(
    "employment_income", map_to="household", period=2026
)

baseline_new_york_mtr_including_health_benefits = simulation_new_york.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

reform_new_york_mtr_including_health_benefits = reformed_simulation_new_york.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

# ---------- Build the graph ----------
fig_new_york_mtr = go.Figure()

# Baseline trace (solid line)
fig_new_york_mtr.add_trace(go.Scatter(
    x=household_income_new_york,
    y=baseline_new_york_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

# Reform trace (dotted line)
fig_new_york_mtr.add_trace(go.Scatter(
    x=household_income_new_york,
    y=reform_new_york_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

# Layout
fig_new_york_mtr.update_layout(
    title='New York Household (Family of 3) – Marginal Tax Rate Including Health Benefits by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Marginal Tax Rate (Including Health Benefits)',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='.0%'),   # assumes MTR is in decimal form (e.g., 0.42 → 42%)
    height=600,
    width=1000
)

# Optional formatting helper if you use one elsewhere
fig_new_york_mtr = format_fig(fig_new_york_mtr)

# Display
fig_new_york_mtr.show()


In [13]:
# ---------- Pull the inputs ----------
household_income_texas = simulation_texas.calculate(
    "employment_income", map_to="household", period=2026
)

baseline_texas_mtr_including_health_benefits = simulation_texas.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

reform_texas_mtr_including_health_benefits = reformed_simulation_texas.calculate(
    "marginal_tax_rate_including_health_benefits",
    map_to="household",
    period=2026
)

# ---------- Build the graph ----------
fig_texas_mtr = go.Figure()

# Baseline trace (solid line)
fig_texas_mtr.add_trace(go.Scatter(
    x=household_income_texas,
    y=baseline_texas_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

# Reform trace (dotted line)
fig_texas_mtr.add_trace(go.Scatter(
    x=household_income_texas,
    y=reform_texas_mtr_including_health_benefits,
    mode='lines',
    name='Marginal Tax Rate (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

# Layout
fig_texas_mtr.update_layout(
    title='Texas Household (Couple) – Marginal Tax Rate Including Health Benefits by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Marginal Tax Rate (Including Health Benefits)',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 200_000]),
    yaxis=dict(tickformat='.0%'),  # assumes rate is in decimal form (0.42 → 42 %)
    height=600,
    width=1000
)

# Optional wrapper if you use one elsewhere
fig_texas_mtr = format_fig(fig_texas_mtr)

# Display
fig_texas_mtr.show()
